<a href="https://colab.research.google.com/github/thuongvan23/XAI-LandCover/blob/nhanLD/Alexnet_satellite__frozen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 81.9 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install albumentations

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tifffile as tiff
import numpy as np
import os

# Đường dẫn đến 1 ảnh cụ thể
img_path = "/kaggle/input/eurosat-dataset/EuroSATallBands/AnnualCrop/AnnualCrop_1.tif"

# Đọc ảnh
img = tiff.imread(img_path)

# In shape và giá trị pixel đầu tiên
print("Shape của ảnh:", img.shape)
print("Pixel đầu tiên:", img[0, 0])

# Chuẩn hóa giá trị pixel trong dải từ 0 đến 1
img_normalized = img / np.max(img)

print(np.max(img))

# Kiểm tra pixel sau khi chuẩn hóa
print("Pixel đầu tiên (sau khi chuẩn hóa):", img_normalized[0, 0])

root_dir = "/kaggle/input/eurosat-dataset/EuroSATallBands"

for root, dirs, files in os.walk(root_dir):
    level = root.replace(root_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files[:5]:
        print(f"{subindent}{f}")

Shape của ảnh: (64, 64, 13)
Pixel đầu tiên: [1420 1291 1291 1600 1680 1841 2045 2016  546   13 3083 2325 2319]
3490
Pixel đầu tiên (sau khi chuẩn hóa): [0.40687679 0.36991404 0.36991404 0.45845272 0.48137536 0.52750716
 0.58595989 0.57765043 0.15644699 0.00372493 0.88338109 0.66618911
 0.66446991]
EuroSATallBands/
  validation.csv
  train.csv
  test.csv
  label_map.json
  SeaLake/
    SeaLake_2893.tif
    Jakarta_000334.tif
    SeaLake_2765.tif
    SeaLake_2733.tif
    SeaLake_1037.tif
  Highway/
    Highway_1963.tif
    Highway_51.tif
    Highway_102.tif
    Highway_1603.tif
    Highway_1224.tif
  River/
    River_173.tif
    River_761.tif
    River_2499.tif
    River_247.tif
    River_926.tif
  Pasture/
    Pasture_666.tif
    Pasture_482.tif
    Pasture_263.tif
    Pasture_1339.tif
    Pasture_690.tif
  Industrial/
    Industrial_2122.tif
    Industrial_725.tif
    Industrial_61.tif
    Industrial_1093.tif
    Industrial_1944.tif
  Residential/
    Residential_1412.tif
    Residenti

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Tạo transform
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.Normalize(mean=0.0, std=1.0, max_pixel_value=1.0),
    ToTensorV2(transpose_mask=True),
])

/usr/local/lib/python3.11/dist-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.5'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [ ]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset
import rasterio
import numpy as np

class EuroSATCSV(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
      row = self.data.iloc[idx]
      img_path = os.path.join(self.root_dir, row['Filename'])
      label = int(row['Label'])

      with rasterio.open(img_path) as src:
          img = src.read().astype(np.float32)  # (13, 64, 64)
          img = img / 10000.0
          img = np.transpose(img, (1, 2, 0))  # (H, W, C) cho albumentations

      if self.transform:
          img = self.transform(image=img)["image"]  # vẫn là (H, W, C)

      img = np.transpose(img, (2, 0, 1))  # Chuyển về (C, H, W) cho PyTorch
      img = torch.tensor(img, dtype=torch.float32)

      return img, label

In [ ]:
base_path = "/kaggle/input/eurosat-dataset/EuroSATallBands"

train_dataset = EuroSATCSV(
    csv_file=os.path.join(base_path, "train.csv"),
    root_dir=base_path,
)

val_dataset = EuroSATCSV(
    csv_file=os.path.join(base_path, "validation.csv"),
    root_dir=base_path,
)

test_dataset = EuroSATCSV(
    csv_file=os.path.join(base_path, "test.csv"),
    root_dir=base_path,
)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision.models import alexnet
from torch.optim.lr_scheduler import ReduceLROnPlateau


class get_finetuned_alexnet(nn.Module):
    def __init__(self, num_classes=10, in_channels=13):
        super(get_finetuned_alexnet, self).__init__()
        # Tạo mô hình AlexNet mà không pretrained
        self.backbone = models.alexnet(pretrained=False)

        # Thay đổi layer conv đầu để nhận 13 kênh
        self.backbone.features[0] = nn.Conv2d(
            in_channels,
            64,
            kernel_size=11,
            stride=4,
            padding=2
        )

        # Đóng băng toàn bộ phần backbone (features)
        for param in self.backbone.features.parameters():
            param.requires_grad = False

        # Thay thế phần classification head (chỉ phần này sẽ được huấn luyện)
        self.backbone.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, delta=0.001):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = float('inf')
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

In [ ]:
# 4. Vòng lặp train/eval
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * inputs.size(0)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)
    return total_loss / total, correct / total

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * inputs.size(0)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)
    return total_loss / total, correct / total

In [ ]:
#5. TRAINING THE MODEL
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_finetuned_alexnet(num_classes=10, in_channels=13).to(device)

# Chỉ train phần classification head (_fc)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
criterion = nn.CrossEntropyLoss()

def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=50):
    early_stopping = EarlyStopping(patience=5, delta=0.001)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

    for epoch in range(epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        scheduler.step(val_loss)
        early_stopping(val_loss)

        print(f"Epoch {epoch+1}/{epochs}:")
        print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")
        print(f"  Learning rate: {optimizer.param_groups[0]['lr']:.6f}")

        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=50)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/50:
  Train Loss: 1.9203 | Train Acc: 0.2817
  Val   Loss: 1.7331 | Val   Acc: 0.3437
  Learning rate: 0.001000
Epoch 2/50:
  Train Loss: 1.6997 | Train Acc: 0.3642
  Val   Loss: 1.5538 | Val   Acc: 0.4385
  Learning rate: 0.001000
Epoch 3/50:
  Train Loss: 1.5658 | Train Acc: 0.4266
  Val   Loss: 1.4135 | Val   Acc: 0.5095
  Learning rate: 0.001000
Epoch 4/50:
  Train Loss: 1.4818 | Train Acc: 0.4515
  Val   Loss: 1.3520 | Val   Acc: 0.5458
  Learning rate: 0.001000
Epoch 5/50:
  Train Loss: 1.4336 | Train Acc: 0.4734
  Val   Loss: 1.2718 | Val   Acc: 0.5459
  Learning rate: 0.001000
Epoch 6/50:
  Train Loss: 1.3767 | Train Acc: 0.4913
  Val   Loss: 1.2189 | Val   Acc: 0.5735
  Learning rate: 0.001000
Epoch 7/50:
  Train Loss: 1.3512 | Train Acc: 0.5053
  Val   Loss: 1.2023 | Val   Acc: 0.5682
  Learning rate: 0.001000
Epoch 8/50:
  Train Loss: 1.3231 | Train Acc: 0.5173
  Val   Loss: 1.1616 | Val   Acc: 0.5863
  Learning rate: 0.001000
Epoch 9/50:
  Train Loss: 1.3040 | Train

In [ ]:
#6. Đánh giá trên tập test
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"\nTest Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")


Test Loss: 0.9796 | Test Acc: 0.6655


In [ ]:
torch.save(model, "/kaggle/working/AlexNet_satellite_frozen.pth")